In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/raw/otodom.csv', index_col=0, na_values='-1')
df.head()

,title,address,price,price_m2,area_m2,rooms,floor,outdoors,parking,market,build_yr,building_type,elevator,heating_type
0,"REZERWACJA Słoneczne, duże, 4 pokoje przy lesie","Gdynia, Pustki Cisowskie-Demptowo, Pustki Ciso...",530 000 zł,6 625 zł/m²,80,4,Piętro\n3/3,balkon,zapytaj,wtórny,1981,blok,nie,miejskie
1,Komfortowe Mieszkanie Port Deco M4,"Gdynia, Oksywie, ul. Benisławskiego",477 037 zł,10 982 zł/m²,"43,44",2,Piętro\nparter/11,balkon,garaż/miejsce parkingowe,pierwotny,2023,brak,tak,zapytaj
2,Apartament z ogródkiem w otulinie lasu,"Gdynia, Orłowo, Bernadowska",1 600 000 zł,24 375 zł/m²,"65,64",3,Piętro\nparter/2,zapytaj,zapytaj,wtórny,2018,brak,nie,zapytaj
3,Elegancki apartament z widokiem na las,"Gdynia, Orłowo, Bernadowska",3 399 000 zł,34 571 zł/m²,"98,32",4,Piętro\n1/2,balkon,garaż/miejsce parkingowe,wtórny,2020,brak,nie,zapytaj
4,Osiedle Beauforta - słoneczne mieszkanie,"Gdynia, Pogórze",522 950 zł,8 900 zł/m²,"58,76",3,Piętro\n1/3,balkon,garaż/miejsce parkingowe,wtórny,2022,apartamentowiec,tak,miejskie


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1927 entries, 0 to 1926
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          1922 non-null   object
 1   address        1922 non-null   object
 2   price          1924 non-null   object
 3   price_m2       1753 non-null   object
 4   area_m2        1925 non-null   object
 5   rooms          1925 non-null   object
 6   floor          1925 non-null   object
 7   outdoors       1925 non-null   object
 8   parking        1925 non-null   object
 9   market         1842 non-null   object
 10  build_yr       1842 non-null   object
 11  building_type  1842 non-null   object
 12  elevator       1842 non-null   object
 13  heating_type   1925 non-null   object
dtypes: object(14)
memory usage: 225.8+ KB


In [4]:
# drop unnecessary columns
df.drop('title', axis=1, inplace=True)

# make some NaN
df = df.replace('zapytaj', np.nan)
df = df.replace('brak', np.nan)

### Clean and convert to number type 

In [5]:
# area
df["area_m2"] = df["area_m2"].str.replace(',','.').astype(float) 

# rooms
df["rooms"] = df['rooms'].apply(lambda x: x if str(x).isdecimal() else np.nan).astype(float)  # string "wiecej" replaced with np.nan

all prices per square meters are in zl/m2

In [6]:
# price per square meters to float 
df['price_m2'] = df['price_m2'].str[:-5].str.replace(' ', '').astype(float)

In [7]:
# total price
df['price'] = df['price'].replace('Zapytaj o cenę', np.nan).str.replace('zł', 'pln').str.replace(' ', '').str.replace(',', '.')

df['currency'] = df.price[~df.price.isna()].apply(lambda x: x[-3:].lower())  # helping column
df['price'] = df.price[~df.price.isna()].apply(lambda x: x[:-3]).astype(float)

In [8]:
df[df['currency'] == 'eur']
# only 3 apartments in dataset has price in eur. But we got price per square meter in pln and apartment area.

,address,price,price_m2,area_m2,rooms,floor,outdoors,parking,market,build_yr,building_type,elevator,heating_type,currency
1055,"Gdynia, Śródmieście, A. Hryniewickiego",1000000.0,38070.0,121.0,4.0,Piętro\n> 10/28,NaN,NaN,wtórny,NaN,NaN,nie,NaN,eur
1462,"Gdynia, Śródmieście, A. Hryniewickiego",1000000.0,38070.0,121.0,4.0,Piętro\n> 10/29,NaN,NaN,wtórny,2009,NaN,nie,NaN,eur
1698,"Gdynia, Orłowo",1000000.0,33996.0,135.5,4.0,Piętro\n1/2,balkon,garaż/miejsce parkingowe,pierwotny,2021,apartamentowiec,tak,gazowe,eur


In [9]:
# change price from eur to pln
df.loc[df['currency'] == 'eur','price'] = df['price_m2'] * df['area_m2']
df.drop('currency', axis=1, inplace=True)  # drop helping column

In [10]:
# floor
def make_floor(series):
    # delete unnecessary text and replace some symbols 
    s = series.str.replace('Piętro\n', '')
    s = s.str.replace("parter", '0').str.replace('suterena', '-0.5').str.replace("> ", '')
    s = s.replace("zapytaj", np.nan)
    s = s.str.replace('poddasze', 'p')

    # for values without max floor in building add / at the end of the string
    s1 = s[~s.isna()]
    s1 = s1[~s1.str.contains('/')].apply(lambda x: x+'/')
    s[s1.index] = s1
    return s

df['floor'] = make_floor(df.floor)

# divide floor column to floor and max floor column 
df['max_floor'] = df['floor'][~df.floor.isna()].apply(lambda x: x.split('/')[1])
df['floor'] = df['floor'][~df.floor.isna()].apply(lambda x: x.split('/')[0])
df = df.replace('', np.nan)

# replace text value with number (p - poddasze)
df.loc[df['floor'] == 'p', 'floor'] = df['max_floor']
df['floor'] = df['floor'].astype(float)
df['max_floor'] = df['max_floor'].astype(float)

In [11]:
# outdoors

# create new columns: balkon, ogrodek, taras
df['balkon'] = df['outdoors'].apply(lambda x: 1 if 'balkon' in str(x) else 0)
df['ogrodek'] = df['outdoors'].apply(lambda x: 1 if 'ogródek' in str(x) else 0)
df['taras'] = df['outdoors'].apply(lambda x: 1 if 'taras' in str(x) else 0)
df.drop('outdoors', axis=1, inplace=True)

In [12]:
# parking
# there is 'garaz/miejsce parkingowe' or NaN -> all NaN will be 0
df['parking'] = df['parking'].apply(lambda x: 1 if str(x) == 'garaż/miejsce parkingowe' else 0)

In [13]:
# from address column create new column with neighbourhood name
df['dzielnica'] = df.loc[~df.address.isna(), 'address'].apply(lambda x: x.split(',')[1])

In [14]:
df.head()

,address,price,price_m2,area_m2,rooms,floor,parking,market,build_yr,building_type,elevator,heating_type,max_floor,balkon,ogrodek,taras,dzielnica
0,"Gdynia, Pustki Cisowskie-Demptowo, Pustki Ciso...",530000.0,6625.0,80.00,4.0,3.0,0,wtórny,1981,blok,nie,miejskie,3.0,1,0,0,Pustki Cisowskie-Demptowo
1,"Gdynia, Oksywie, ul. Benisławskiego",477037.0,10982.0,43.44,2.0,0.0,1,pierwotny,2023,NaN,tak,NaN,11.0,1,0,0,Oksywie
2,"Gdynia, Orłowo, Bernadowska",1600000.0,24375.0,65.64,3.0,0.0,0,wtórny,2018,NaN,nie,NaN,2.0,0,0,0,Orłowo
3,"Gdynia, Orłowo, Bernadowska",3399000.0,34571.0,98.32,4.0,1.0,1,wtórny,2020,NaN,nie,NaN,2.0,1,0,0,Orłowo
4,"Gdynia, Pogórze",522950.0,8900.0,58.76,3.0,1.0,1,wtórny,2022,apartamentowiec,tak,miejskie,3.0,1,0,0,Pogórze


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1927 entries, 0 to 1926
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   address        1922 non-null   object 
 1   price          1752 non-null   float64
 2   price_m2       1753 non-null   float64
 3   area_m2        1925 non-null   float64
 4   rooms          1923 non-null   float64
 5   floor          1909 non-null   float64
 6   parking        1927 non-null   int64  
 7   market         1842 non-null   object 
 8   build_yr       1608 non-null   object 
 9   building_type  1361 non-null   object 
 10  elevator       1842 non-null   object 
 11  heating_type   1303 non-null   object 
 12  max_floor      1826 non-null   float64
 13  balkon         1927 non-null   int64  
 14  ogrodek        1927 non-null   int64  
 15  taras          1927 non-null   int64  
 16  dzielnica      1922 non-null   object 
dtypes: float64(6), int64(4), object(7)
memory usage: 335

In [16]:
df.to_csv('../data/interim/otodom_interim.csv')